In [1]:
import nest_asyncio
from flask import Flask, render_template_string, request
import nltk
import numpy as np
import re
from nltk.corpus import wordnet as wn
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import RegexpParser
import random
from itertools import chain

# Apply the asyncio patch to allow Flask to run inside Jupyter
nest_asyncio.apply()

# Initialize Flask app
app = Flask(__name__)

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Grammar for chunking
CHUNK_GRAMMAR = r"""
    CHUNK: {<NN>+<IN|DT>*<NN>+}
           {<NN>+<IN|DT>*<NNP>+}
           {<NNP>+<NNS>*}
"""
chunk_parser = RegexpParser(CHUNK_GRAMMAR)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\dsoni\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
class SubjectiveTest:
    def __init__(self, data, no_of_ques):
        self.data = data
        self.no_of_ques = no_of_ques
        self.question_pattern = [
            "Explain in detail how ",
            "Describe the significance of ",
            "What are the key aspects of ",
            "Discuss the role of ",
            "Provide an overview of "
        ]

    def generate_test(self):
        sentences = sent_tokenize(self.data)
        question_answer = []
        used_keywords = set()

        for sentence in sentences:
            # Extract meaningful keywords (nouns, proper nouns)
            tagged_words = pos_tag(word_tokenize(sentence))
            chunks = []
            for word, pos in tagged_words:
                if pos in {"NN", "NNS", "NNP", "NNPS"}:
                    chunks.append(word.lower())

            # Try generating multiple questions from the same sentence using distinct keywords
            for i in range(len(chunks)):
                # Stop if we've reached the desired number of questions
                if len(question_answer) >= self.no_of_ques:
                    break

                # Create a keyword phrase using up to 3 consecutive nouns starting at position i
                keyword = " ".join(chunks[i:i+3])
                if keyword and keyword not in used_keywords:
                    used_keywords.add(keyword)
                    pattern = np.random.randint(0, len(self.question_pattern))
                    question = self.question_pattern[pattern] + keyword + "?"
                    question_answer.append({
                        "Question": question,
                        "Answer": sentence
                    })

            # Stop processing more sentences if we've reached required questions
            if len(question_answer) >= self.no_of_ques:
                break

        return question_answer
    
class ObjectiveTest:
    def __init__(self, data, no_of_ques):
        self.data = data
        self.no_of_ques = no_of_ques
        self.used_distractors = set()  # Track all used distractors
        # Define chunking pattern for noun phrases
        self.chunk_pattern = """
            CHUNK: {<DT>?<JJ>*<NN.*>+}
                   {<NNP>+}
                   {<JJ>*<NN.*>}
        """
        self.chunk_parser = RegexpParser(self.chunk_pattern)

    def get_wordnet_relationships(self, word):
        """Get related words using various WordNet relationships"""
        related_words = set()
        
        # Get all synsets for the word
        synsets = wn.synsets(word)
        for synset in synsets:
            # Add lemmas from the same synset (synonyms)
            related_words.update(lemma.name() for lemma in synset.lemmas())
            
            # Add hypernyms (more general terms)
            related_words.update(chain.from_iterable(
                lemma.name() for hyper in synset.hypernyms() 
                for lemma in hyper.lemmas()
            ))
            
            # Add hyponyms (more specific terms)
            related_words.update(chain.from_iterable(
                lemma.name() for hypo in synset.hyponyms() 
                for lemma in hypo.lemmas()
            ))
            
            # Add sister terms (share a hypernym)
            for hypernym in synset.hypernyms():
                for hyponym in hypernym.hyponyms():
                    related_words.update(lemma.name() for lemma in hyponym.lemmas())
            
            # Add meronyms (part-of relationships)
            for meronym in synset.part_meronyms():
                related_words.update(lemma.name() for lemma in meronym.lemmas())
            
            # Add holonyms (whole-of relationships)
            for holonym in synset.part_holonyms():
                related_words.update(lemma.name() for lemma in holonym.lemmas())

        return {word.replace('_', ' ') for word in related_words}

    def get_smart_distractors(self, phrase, context_sentence):
        """
        Generate unique distractors using multiple strategies
        """
        all_distractors = set()
        phrase_lower = phrase.lower()
        
        # Strategy 1: WordNet relationships for each word in the phrase
        words = phrase_lower.split()
        for word in words:
            related_words = self.get_wordnet_relationships(word)
            # Create phrase variations by replacing each word
            for related in related_words:
                if related.lower() != word:
                    new_phrase = phrase.replace(word, related)
                    if new_phrase.lower() != phrase_lower:
                        all_distractors.add(new_phrase.title())

        # Strategy 2: Part-of-Speech based variations
        tagged_words = pos_tag(word_tokenize(context_sentence))
        similar_pos_phrases = []
        target_pos = pos_tag(word_tokenize(phrase))
        target_pattern = [tag for _, tag in target_pos]
        
        # Find phrases with similar POS pattern
        i = 0
        while i < len(tagged_words) - len(target_pattern) + 1:
            current_pattern = [tag for _, tag in tagged_words[i:i+len(target_pattern)]]
            if current_pattern == target_pattern:
                candidate = ' '.join(word for word, _ in tagged_words[i:i+len(target_pattern)])
                if candidate.lower() != phrase_lower:
                    similar_pos_phrases.append(candidate)
            i += 1

        all_distractors.update(similar_pos_phrases)

        # Strategy 3: Length-based variations
        if len(all_distractors) < 3:
            words = phrase.split()
            if len(words) > 1:
                # Reverse word order
                all_distractors.add(' '.join(words[::-1]))
                # Rotate words
                all_distractors.add(' '.join(words[1:] + [words[0]]))
                # Add/remove modifiers
                common_modifiers = ['New', 'Advanced', 'Basic', 'Modern']
                for modifier in common_modifiers:
                    if not phrase.startswith(modifier):
                        all_distractors.add(f"{modifier} {phrase}")

        # Strategy 4: Structural variations
        if len(all_distractors) < 3:
            # Handle numeric variations if present
            if any(char.isdigit() for char in phrase):
                numbers = re.findall(r'\d+', phrase)
                for num in numbers:
                    num_int = int(num)
                    variants = [num_int * 2, num_int + 10, num_int - 5]
                    for variant in variants:
                        if variant > 0:
                            all_distractors.add(phrase.replace(num, str(variant)))

        # Remove any distractors that are too similar to the correct answer
        all_distractors = {d for d in all_distractors 
                          if not self._is_too_similar(d.lower(), phrase_lower)}

        # Remove previously used distractors
        all_distractors = all_distractors - self.used_distractors

        # Convert to list and get top 3 most different distractors
        distractors = list(all_distractors)
        if len(distractors) > 3:
            # Sort by difference from correct answer to get most distinct options
            distractors.sort(key=lambda x: self._levenshtein_distance(x.lower(), phrase_lower), reverse=True)
            distractors = distractors[:3]
        
        # Add to used distractors
        self.used_distractors.update(distractors)

        # If we still don't have enough distractors, generate some
        while len(distractors) < 3:
            new_distractor = self._generate_fallback_distractor(phrase, len(distractors))
            if new_distractor not in self.used_distractors:
                distractors.append(new_distractor)
                self.used_distractors.add(new_distractor)

        return distractors

    def _is_too_similar(self, str1, str2, threshold=0.8):
        """Check if two strings are too similar using Levenshtein distance"""
        distance = self._levenshtein_distance(str1, str2)
        max_len = max(len(str1), len(str2))
        similarity = 1 - (distance / max_len)
        return similarity > threshold

    def _levenshtein_distance(self, s1, s2):
        """Calculate the Levenshtein distance between two strings"""
        if len(s1) < len(s2):
            return self._levenshtein_distance(s2, s1)
        if len(s2) == 0:
            return len(s1)
        previous_row = range(len(s2) + 1)
        for i, c1 in enumerate(s1):
            current_row = [i + 1]
            for j, c2 in enumerate(s2):
                insertions = previous_row[j + 1] + 1
                deletions = current_row[j] + 1
                substitutions = previous_row[j] + (c1 != c2)
                current_row.append(min(insertions, deletions, substitutions))
            previous_row = current_row
        return previous_row[-1]

    def _generate_fallback_distractor(self, phrase, index):
        """Generate a fallback distractor when other methods don't provide enough options"""
        words = phrase.split()
        if len(words) == 1:
            # For single words, modify the word structure
            modifications = [
                phrase + 's' if not phrase.endswith('s') else phrase[:-1],
                phrase[::-1].title(),  # Reverse the word
                phrase + str(index + 1)  # Add a number
            ]
            return modifications[index % len(modifications)]
        else:
            # For phrases, modify the structure
            modifications = [
                ' '.join(words[::-1]),  # Reverse word order
                ' '.join(sorted(words)),  # Alphabetically sort words
                ' '.join(words + [str(index + 1)])  # Add a number
            ]
            return modifications[index % len(modifications)]

    def generate_test(self):
        sentences = sent_tokenize(self.data)
        question_answer = []
        used_phrases = set()

        for sentence in sentences:
            # Continue generating questions until we reach the desired count
            if len(question_answer) >= self.no_of_ques:
                break

            tagged_words = pos_tag(word_tokenize(sentence))
            tree = self.chunk_parser.parse(tagged_words)

            for subtree in tree.subtrees():
                if subtree.label() == "CHUNK":
                    phrase = " ".join(word for word, _ in subtree.leaves())
                    
                    if phrase not in used_phrases and len(phrase.split()) <= 3:
                        used_phrases.add(phrase)
                        blanks = "_" * len(phrase)
                        question = re.sub(re.escape(phrase), blanks, sentence, count=1)
                        
                        # Get smart distractors using the entire sentence for context
                        distractors = self.get_smart_distractors(phrase, sentence)
                        
                        # Combine correct answer and distractors
                        options = [phrase] + distractors
                        
                        # Shuffle options
                        random.shuffle(options)
                        
                        question_answer.append({
                            "Question": question,
                            "Answer": phrase,
                            "Options": options
                        })
                        
                        # Check if we've met the required number of questions after adding one
                        if len(question_answer) >= self.no_of_ques:
                            break
            # Optional: Check again after finishing sentence chunks
            if len(question_answer) >= self.no_of_ques:
                break

        return question_answer


In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
    generated_questions = []
    input_text = ""
    no_of_questions = 0
    test_type = "subjective"

    if request.method == 'POST':
        input_text = request.form['input_text']
        no_of_questions = int(request.form['no_of_questions'])
        test_type = request.form['test_type']

        if test_type == 'subjective':
            test = SubjectiveTest(input_text, no_of_questions)
        elif test_type == 'objective':
            test = ObjectiveTest(input_text, no_of_questions)
        
        generated_questions = test.generate_test()

    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
        <title>Q/A Generator</title>
        <!-- Bootstrap CSS -->
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
        <!-- Font Awesome -->
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
        <style>
            :root {
                --primary-color: #4361ee;
                --secondary-color: #3f37c9;
            }
            
            body {
                background-color: #f8f9fa;
                padding-top: 4.5rem;
            }
            
            .navbar {
                background: linear-gradient(135deg, var(--primary-color), var(--secondary-color));
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            
            .navbar-brand {
                font-weight: 600;
                display: flex;
                align-items: center;
            }
            
            .card {
                border: none;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);
                margin-bottom: 1.5rem;
                transition: transform 0.2s ease;
            }
            
            .card:hover {
                transform: translateY(-2px);
            }
            
            .form-control {
                border-radius: 8px;
                border: 1px solid #ced4da;
                padding: 0.75rem;
            }
            
            .form-control:focus {
                border-color: var(--primary-color);
                box-shadow: 0 0 0 0.2rem rgba(67, 97, 238, 0.25);
            }
            
            .btn-primary {
                background-color: var(--primary-color);
                border: none;
                border-radius: 8px;
                padding: 0.75rem 1.5rem;
                font-weight: 600;
                transition: all 0.3s ease;
            }
            
            .btn-primary:hover {
                background-color: var(--secondary-color);
                transform: translateY(-1px);
            }
            
            .question-card {
                border-left: 4px solid var(--primary-color);
            }
            
            .options-grid {
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 1rem;
                margin-top: 1rem;
            }
            
            .option-item {
                background-color: #f8f9fa;
                padding: 0.75rem;
                border-radius: 8px;
                font-weight: 500;
            }

            .loading {
                position: relative;
                pointer-events: none;
            }
            
            .loading:after {
                content: '';
                position: absolute;
                width: 16px;
                height: 16px;
                top: 0;
                left: 0;
                right: 0;
                bottom: 0;
                margin: auto;
                border: 3px solid #ffffff;
                border-top-color: transparent;
                border-radius: 50%;
                animation: spin 1s ease infinite;
            }
            
            @keyframes spin {
                from { transform: rotate(0deg); }
                to { transform: rotate(360deg); }
            }
        </style>
    </head>
    <body>
        <!-- Navigation Bar -->
        <nav class="navbar navbar-expand-md navbar-dark fixed-top">
            <div class="container">
                <a class="navbar-brand" href="#">
                    <i class="fas fa-brain mr-2"></i>
                    Q/A Generator
                </a>
            </div>
        </nav>

        <div class="container">
            <div class="card mt-4 p-4">
                <h1 class="h3 mb-4">
                    <i class="fas fa-magic mr-2 text-primary"></i>
                    Generate Your Questions
                </h1>
                <form method="POST" id="questionForm">
                    <div class="form-group">
                        <label for="input_text">
                            <i class="fas fa-file-alt mr-2"></i>Input Text
                        </label>
                        <textarea 
                            class="form-control" 
                            id="input_text" 
                            name="input_text" 
                            rows="6" 
                            placeholder="Paste your text here..."
                            required
                        >{{ input_text }}</textarea>
                    </div>
                    <div class="row">
                        <div class="col-md-6">
                            <div class="form-group">
                                <label for="no_of_questions">
                                    <i class="fas fa-list-ol mr-2"></i>Number of Questions
                                </label>
                                <input 
                                    type="number" 
                                    class="form-control" 
                                    id="no_of_questions" 
                                    name="no_of_questions" 
                                    value="{{ no_of_questions }}" 
                                    min="1" 
                                    required
                                >
                            </div>
                        </div>
                        <div class="col-md-6">
                            <div class="form-group">
                                <label for="test_type">
                                    <i class="fas fa-tasks mr-2"></i>Test Type
                                </label>
                                <select class="form-control" id="test_type" name="test_type" required>
                                    <option value="subjective" {% if test_type == 'subjective' %}selected{% endif %}>
                                        Subjective
                                    </option>
                                    <option value="objective" {% if test_type == 'objective' %}selected{% endif %}>
                                        Objective
                                    </option>
                                </select>
                            </div>
                        </div>
                    </div>
                    <button type="submit" class="btn btn-primary btn-block">
                        <i class="fas fa-wand-magic-sparkles mr-2"></i>
                        Generate Questions
                    </button>
                </form>
            </div>

            {% if questions %}
                <h2 class="h4 mt-5 mb-4">
                    <i class="fas fa-clipboard-list mr-2 text-primary"></i>
                    Generated {{ test_type.capitalize() }} Questions
                </h2>
                <div class="questions-container">
                    {% for q in questions %}
                        <div class="card question-card">
                            <div class="card-body">
                                <h5 class="card-title text-muted small mb-2">Question {{ loop.index }}</h5>
                                <p class="card-text h5 mb-4">{{ q.Question }}</p>
                                {% if test_type == 'objective' %}
                                    <h6 class="text-muted small mb-2">Options</h6>
                                    <div class="options-grid">
                                        {% for option in q.Options %}
                                            <div class="option-item">{{ option }}</div>
                                        {% endfor %}
                                    </div>
                                {% endif %}
                                <div class="mt-4">
                                    <h6 class="text-muted small mb-2">Answer</h6>
                                    <p class="mb-0">{{ q.Answer }}</p>
                                </div>
                            </div>
                        </div>
                    {% endfor %}
                </div>
            {% endif %}
        </div>

        <!-- Bootstrap JS, Popper.js, and jQuery -->
        <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js"></script>
        <script src="https://cdn.jsdelivr.net/npm/popper.js@1.16.1/dist/umd/popper.min.js"></script>
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
        <script>
            // Add loading state to form submission
            document.getElementById('questionForm').addEventListener('submit', function(e) {
                const button = this.querySelector('button[type="submit"]');
                const icon = button.querySelector('i');
                const text = button.textContent.trim();
                
                button.classList.add('loading');
                button.innerHTML = '<span style="opacity: 0;">Generating...</span>';
                
                // Re-enable after 100ms if form is invalid
                setTimeout(() => {
                    if (!this.checkValidity()) {
                        button.classList.remove('loading');
                        button.innerHTML = `<i class="${icon.className}"></i> ${text}`;
                    }
                }, 100);
            });
        </script>
    </body>
    </html>
    """
    return render_template_string(html_template, questions=generated_questions, input_text=input_text, no_of_questions=no_of_questions, test_type=test_type)

# Run Flask application
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port=5001)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [07/Jan/2025 16:17:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 16:18:04] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 16:18:16] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 16:20:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 16:20:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jan/2025 16:20:36] "POST / HTTP/1.1" 200 -
